# Test Modules

In [ ]:
# Set project root directory and add `src` to path
import sys
from pathlib import Path

PROJECT_ROOT = '/scratch/edk202/word2gm-fast'
project_root = Path(PROJECT_ROOT)
src_path = project_root / 'src'

if str(src_path) not in sys.path:
    sys.path.insert(0, str(src_path))

# Import the notebook setup utilities
from word2gm_fast.utils.notebook_setup import setup_testing_notebook, enable_autoreload, run_silent_subprocess

# Enable autoreload for development
enable_autoreload()

# Set up environment
env = setup_testing_notebook(project_root=PROJECT_ROOT)

# Extract commonly used modules for convenience
tf = env['tensorflow']
np = env['numpy']
pd = env['pandas']
print_resource_summary = env['print_resource_summary']

: 

In [ ]:
import datetime

modules_to_test = [
    ('tests/test_corpus_to_dataset.py', 'Corpus processing'),
    ('tests/test_index_vocab.py', 'Vocabulary indexing'), 
    ('tests/test_dataset_to_triplets.py', 'Triplet generation'),
    ('tests/test_tfrecord_io.py', 'TFRecord I/O'),
    ('tests/test_word2gm_model.py', 'Word2GM model')
]

output_lines = ["Testing modules..."]
failures = []

for test_file, test_name in modules_to_test:
    start_time = datetime.datetime.now()
    cmd_result = run_silent_subprocess(
        ['pytest', test_file, '--maxfail=1', '--disable-warnings', '-q'],
        capture_output=True, text=True
    )
    status = "PASS" if cmd_result.returncode == 0 else "FAIL"
    output_lines.append(f"{test_name}: {status}")
    if cmd_result.returncode != 0:
        failures.append((test_name, cmd_result))

print("\n".join(output_lines))

if failures:
    print("\nDetailed output for failed tests:")
    for test_name, cmd_result in failures:
        print(f"\n--- {test_name} ---")
        if cmd_result.stdout:
            print("stdout:\n" + cmd_result.stdout)
        if cmd_result.stderr:
            print("stderr:\n" + cmd_result.stderr)